# Preparing CONUS404 and reference data for aggregation and sampling

<img src='./Eval_PreProc.svg' width=600>
The purpose of data preparation notebooks is to process forcing and reference datasets for comparison and evaluation. Each preparation notebook will differ somewhat in its specifications; for instance, by forcing dataset, reference datasets, variables, temporal resolution, spatial extent, etc.

Specifications for this notebook:
* Forcing data: CONUS404
* Reference datasets: PRISM (gridded), CERES-EBAF (gridded), GHCN (point), USCRN (point)
* Variables: precipitation, mean temperature, radiation (to add: tmin, tmax, radiation components)
* Processed temporal resolution: monthly
* Processed spatial extent: DRB, by HUC6

These are the steps this notebook follows:
* Step 0: Import libraries
* Step 1: Retrieve data from the HPC or Cloud
* Step 2: Explore the data
* Step 3: Import geographic extents
* Step 4: Put it together: Process CONUS404 to variable and spatial extent
* Step 5: Prepare gridded reference data
* Step 6: Prepare station reference data

## **Step 0: Importing libraries**

In [ ]:
# library imports
import fsspec
import hvplot.xarray
import intake
import os
import warnings
import rioxarray
import dask
import metpy
import calendar
import hvplot.pandas  

from shapely.geometry import Polygon
from dask.distributed import LocalCluster, Client
import pygeohydro
from fsspec.implementations.ftp import FTPFileSystem
from holoviews.streams import PolyEdit, PolyDraw

import xarray as xr
import geopandas as gpd
import pandas as pd
import geoviews as gv
import dask.dataframe as dd
import numpy as np

warnings.filterwarnings('ignore')

# **Step 1: Retrieving data from HPC or the Cloud**
#### The process varies based on where the notebook is being run but generally looks this:
1.  Connect to workspace (local, HPC, or QHUB) and open notebook **(Done already)**
2. Start Dask client 
3. Pull in data from source
4. Process the data into usable file format, size, and extent

### Start a Dask client using an appropriate Dask Cluster
This is an optional step, but can speed up data loading significantly, especially when accessing data from the cloud.

In [ ]:
def configure_cluster(machine):
    ''' Helper function to configure cluster
    '''
    if machine == 'denali':
        from dask.distributed import LocalCluster, Client
        cluster = LocalCluster(threads_per_worker=1)
        client = Client(cluster)
    
    elif machine == 'tallgrass':
        from dask.distributed import Client
        from dask_jobqueue import SLURMCluster
        cluster = SLURMCluster(queue='cpu', cores=1, interface='ib0',
                               job_extra=['--nodes=1', '--ntasks-per-node=1', '--cpus-per-task=1'],
                               memory='6GB')
        cluster.adapt(maximum_jobs=30)
        client = Client(cluster)
        
    elif machine == 'local':
        import os
        import warnings
        from dask.distributed import LocalCluster, Client
        warnings.warn("Running locally can result in costly data transfers!\n")
        n_cores = os.cpu_count() # set to match your machine
        cluster = LocalCluster(threads_per_worker=n_cores)
        client = Client(cluster)
        
    elif machine in ['esip-qhub-gateway-v0.4']:   
        import sys, os
        sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
        import ebdpy as ebd
        aws_profile = 'esip-qhub'  
        ebd.set_credentials(profile=aws_profile)

        aws_region = 'us-west-2'
        endpoint = f's3.{aws_region}.amazonaws.com'
        ebd.set_credentials(profile=aws_profile, region=aws_region, endpoint=endpoint)
        worker_max = 30
        client,cluster = ebd.start_dask_cluster(profile=aws_profile, worker_max=worker_max, 
                                              region=aws_region, use_existing_cluster=True,
                                              adaptive_scaling=True, wait_for_cluster=False, 
                                              worker_profile='Medium Worker', propagate_env=True)
        
    return client, cluster

### View available datasets from the Intake Catalog and choose which to use
Note: Select datasets that end in "onprem" if running on Denali/Tallgrass HPC or cloud data if working on Nebari or local.

In [ ]:
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'
cat = intake.open_catalog(url)
list(cat)

### You can setup your client and dataset on Nebari or HPC like this:

In [ ]:
if 'SLURM_CLUSTER_NAME' in os.environ: #USGS HPC use SLURM CLUSTER to handle jobs, otherwise...
    dataset = 'conus404-hourly-onprem'
    machine = os.environ['SLURM_CLUSTER_NAME']
    cluster = configure_cluster(machine)
else:  # use the Nebari machine
    dataset = 'conus404-hourly-cloud'
    machine = 'esip-qhub-gateway-v0.4'
    client, cluster = configure_cluster(machine)

### Retrieve CONUS404 from source and tranform it to a Dask array

In [ ]:
# double check that dataset is in catalog (cat)
dataset = 'conus404-hourly-cloud'
cat[dataset]

Transform the data to a Dask Dataset using the `to_dask()` method from the `intake` package.

In [ ]:
ds = cat[dataset].to_dask()

View dataset metadata. To view variables, expand the *Data variables* section and a greater explanation of the data variables can be found on the [HyTEST SharePoint](https://doimspp.sharepoint.com/sites/gs-wma-hytest/SitePages/CONUS404-Initial-Variables.aspx?source=https%3a//doimspp.sharepoint.com/sites/gs-wma-hytest/SitePages/Forms/ByAuthor.aspx)

For this tutorial, we will be working with the accumulated precipitation (PREC_ACC_NC), air temperature (TK), and surface net radiation (RNET) variables. If you look through the *Data variables* section of *ds*, you'll notice RNET doesn't exist. It will be calculated using other existing variables later in the tutorial.

In [ ]:
ds

## **Step 2: Explore the data** 
(sometimes called exploratory data analysis (EDA) or exploratory spatial data analysis (ESDA) when it contains cartographic data)

### Let's look at the accumulated precipitation variable by first subsetting the larger dataset. 

This makes the data lighter and working with it more responsive. <br>

Notice the information in the array and chunk columns as well as the coordinates (in particular *time*) and the units.

In [ ]:
# variable PREC_ACC_NC
prec = ds.PREC_ACC_NC
prec

### Next, lets visualize a map of the data at specific time step.

In [ ]:
prec_time = prec.sel(time='2014-03-01 00:00').load()

In the previous cell, the .sel() method filters the dataset by the *time* coordinate through "time=" and then uses the .load() method to load the dataset into memory. More on indexing datasets can be found at [Indexing and selecting data](https://docs.xarray.dev/en/stable/user-guide/indexing.html).

Now, let's visualize the dataset using the [QuadMesh](https://holoviews.org/reference/elements/bokeh/QuadMesh.html) plot from Holoviews. For a more in-depth tutorial for visualizing gridded data in Holoviews, go to [Gridded Datasets](http://holoviews.org/getting_started/Gridded_Datasets.html).

Notice the spatial extent and the pattern of values.

In [ ]:
prec_time.hvplot.quadmesh(x='lon', y='lat', rasterize=True, geo=True, tiles='OSM', alpha=0.7, cmap='turbo')

### We can also look at a time-series for a specific grid cell.

In [ ]:
prec_point = prec.isel(y=600,x=600).sel(time=slice('2015-02-11 00:00','2015-04-28 00:00')).load()

Note the previous cell uses the .isel() method, which returns the dataset from where the **x** and **y** indexes equal 600 prior to filtering by **time** and loading the data into memory. Essentially, we're plotting a time series at the grid cell whose coordinates are x=y=600, and we're also defining a period of time to look at through definition of the time slice.

Lets plot the dataset.

In [ ]:
prec_point.hvplot(x='time', grid=True)

## **Step 3: Importing geographic extents**
Sometimes geographic data is brought in and used to clip a larger dataset to an area of interest (AOI). 

Let's look at two ways this can be done: a user-defined polygon or using the pyNHD package. Data can also be brought in other ways such as a local file or an API request. These are covered in other tutorials such as [Reading and Writing Files (GeoPandas)](https://geopandas.org/en/stable/docs/user_guide/io.html) or using the Python [requests](https://requests.readthedocs.io/en/latest/) package for [Programattically Accesing Geospatial Data Using APIs](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/spatial-data-using-apis/).

We'll show how to use geometries to spatially clip datasets later in this tutorial.

### The first method will use the the Holoviews and Geoviews libraries to draw a polygon and then add its dimensions to a geopandas GeoDataFrame.

When the next code block is run, a map will open and the PolyDraw tool will be automatically selected. 

**Double-click to add the first vertex, then single-click to add each subsequent vertex, and  to finalize the draw action double-click to insert the final vertex or press the ESC key to stop drawing after adding the last vertex.**

In [ ]:
#use CartoLight basemap
basemap = gv.tile_sources.CartoLight()

# x and y limits for CONUS
xlim = (-135, -50)
ylim = (22, 50)

#create blank polygon to draw
## redim.range works with Bokeh backend to set default map extent
blank_poly = gv.Polygons([]).redim.range(Longitude=xlim, Latitude=ylim)

# set PolyDraw for creation and PolyEdit for editing polygon, num_objects keeps to single object at a time
user_poly = PolyDraw(source=blank_poly, show_vertices=True, num_objects=1)
user_poly_edit = PolyEdit(source=blank_poly)

# create plots
## active_tools set to allow instant polygon drawing
basemap.options(width=700, height=400) * blank_poly.options(
    active_tools=['poly_draw'], fill_alpha=0.2, line_color='black')

The next code block pulls the latitude and longitude coordinates for the polygon vertices that were just drawn and creates a `geopandas` GeoDataFrame object. More info about `geopandas` [can be found here](https://geopandas.org/en/stable/docs/user_guide.html).

In [ ]:
# extract lists of lat/long coordinates
long = user_poly.data['xs'][0]
lat = user_poly.data['ys'][0]

# create zip of polygon vertices
vertices = zip(long, lat)

# construct polygon in GDF
polygon = gpd.GeoDataFrame(
    index=[0], crs="EPSG:4326", geometry=[Polygon(vertices)])

In [ ]:
# plot polygon to confirm the shape matches what was drawn
polygon.plot()

The polygon can then be using to clip rasters or other vector data.

### The second method will be importing the HUC6 boundaries using the PyGeoHydro library. PyGeoHydro is a part of the HyRiver family of libraries and is documented [here](https://docs.hyriver.io/autoapi/pygeohydro/index.html).

The following cell queries the Water Boundary Dataset HUC6 layer and returns a GeoDataFrame from the .byids() function by examining the "huc6" field (first argument) for the list of HUC6 id's (second argument). The two HUC6 ids used here correspond to the Delware River Basin (DRB).

In [ ]:
drb = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])
drb

You see there are two polygons in the GeoDataFrame and plotting them confirms this.

In [ ]:
drb.plot(edgecolor="orange", facecolor="purple", linewidth=2.5)

When you want to use geometries to refine datasets to an AOI, it is best to have a single, concise geometry. We'll combine the two polygons in the next code cell.

In [ ]:
# create a column where all entries have the same value
drb["name"] = "DRB"

#dissolve by that column
drb = drb.dissolve(by="name")

Check to make sure it worked by examing the tabular and spatial data.

In [ ]:
# tabular 
drb.head()

In [ ]:
# spatial
drb.plot(edgecolor="orange", facecolor="purple", linewidth=2.5)

## **Step 4: Putting it together: Process CONUS404 to variable and spatial extent**
In this section we are going to put together some skills we have learned so far: bring in CONUS404, select our variables, then clip to our spatial extent. This assumes that the notebook is being run on the ESIP QHub. If being run on HPC then comment/uncomment the datasets as needed.

Variables: Accumulated precipitation (PREC_ACC_NC), air temperature (TK), and (calculated) surface net radiation (RNET) <br>
Spatial extent: Delaware River Basin (DRB)<br>

In [ ]:
# set up conus404 filename
conus404 = 'conus404-hourly-cloud' #ESIP QHub
# conus404 = 'conus404-hourly-onprem' #HPC

# create dask array from dataset
ds = cat[conus404].to_dask()

# copy crs data variable for later
ds_crs_array = ds.variables["crs"]

# parse spatial information from CF conventions
ds = ds.metpy.parse_cf()

Get the coordinate reference system (CRS) information from CONUS404 dataset to use when setting the CRSs for other datasets

In [ ]:
crs = ds["TK"].metpy.cartopy_crs
# crs

Reference datasets that that are brought in might need to be sliced to the same time period as the CONUS404 dataset. Set the start and end dates for the CONUS404 dataset to use later.

In [ ]:
# get the minimum time coordinate
start_date = ds.coords["time"].values.min()

# convert to datetime then extract the year-month as a string "YYYYmm"
start_date = pd.to_datetime(start_date).strftime("%Y-%m")

# add the first day of the month to the date
start_date = f"{start_date}-01"

# get the maximum time coordinate
end_date = ds.coords["time"].values.max()

# convert to datetime then extract the year-month as a string "YYYYmm"
end_date = pd.to_datetime(end_date).strftime("%Y-%m")

# as the end date of months vary, use the monthrange function, which returns a tuple of integers as (firstDay, lastDay)
# extract the lastDay integer by indexing [1] and convert it to string
last_day = calendar.monthrange(int(end_date[0:4]), int(end_date[-2:]))[1]
last_day = str(last_day) 

# add last_data to end_date                         
end_date = f"{end_date}-{last_day}"

print("Start date:", start_date, "\nEnd date:", end_date)

Create a subset of desired data variables and then subset the variables:
<table align="left">
  <tr>
    <th>Short name</th>
    <th>Long name</th>
  </tr>
  <tr>
    <td>PREC_ACC_NC</td>
    <td>accumulated prescipitation</td>
  </tr>
  <tr>
    <td>TK</td>
    <td>average temperature</td>
  </tr>
  <tr>
    <td>ACSWDNB</td>
    <td>accumulated downwelling surface flux of shortwave radiation</td>
  </tr>
  <tr>
    <td>ACSWUPB</td>
    <td>accumulated upwelling surface flux of shortwave radiation</td>
  </tr>
  <tr>
    <td>ACLWDNB</td>
    <td>accumulated downwelling surface flux of longwave radiation</td>
  </tr>
  <tr>
    <td>ACLWUPB</td>
    <td>accumulated upwelling surface flux of longwave radiation</td>
  </tr>
</table>

In [ ]:
#subset data variables
c404_variables = ["PREC_ACC_NC", "TK", "ACSWDNB", "ACSWUPB", "ACLWDNB", "ACLWUPB", "crs"]
c404 = ds[c404_variables]

Now bring in our AOI boundaries and as before, we will use the DRB HUC6 polygons. We will calculate the bounding box of the DRB, which will be used in the next step to pare the CONUS404 data down spatially.

In [ ]:
# bring in boundaries of DRB and create single polygon
drb = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])
# create a column where all entries have the same value
drb["name"] = "DRB"

# dissolve by that column
drb = drb.dissolve(by="name")

# set CRS to match ds
drb = drb.iloc[[0]].to_crs(crs)

# tuple of bounding box
drb_bbox = list(drb.total_bounds)

#visualize
# drb.plot(edgecolor="orange", facecolor="purple", linewidth=2.5)

Now clip by the bounding box.

In [ ]:
# write CRS
c404.rio.write_crs(crs, inplace=True)

# perform clip by bounding box
c404_drb = c404.rio.clip_box(minx=drb_bbox[0],
                            miny=drb_bbox[1],
                            maxx=drb_bbox[2],
                            maxy=drb_bbox[3],
                            crs=crs)

Visualize the results

In [ ]:
c404_drb["TK"].isel(time=-1).hvplot(x='x', y='y', crs=crs, geo=True, rasterize=True, cmap='turbo', tiles='OSM', alpha=0.5)

We have a little more processing to do before the dataset is ready for analysis. We need to:
1. Calcuate RNET using the radiation columns
2. Resample and aggregate the data from the input hourly timestep to the desired time-step (1 month)
3. Perform unit conversion to align with desired units

RNET is calculated using this equation:<br><br><br>
\begin{equation}RNET = SWDN + LWDN - SWUP - LWUP\end{equation}

and those variables are detailed below.

<table align="left" style="page-break-after: always;">
  <tr>
    <th>Short name</th>
    <th>Long name</th>
  </tr>
  <tr>
    <td>SWDN</td>
    <td>downwelling surface flux of shortwave radiation</td>
  </tr>
  <tr>
    <td>LWDN</td>
    <td>downwelling surface flux of longwave radiation</td>
  </tr>
  <tr>
    <td>SWUP</td>
    <td>upwelling surface flux of shortwave radiation</td>
  </tr>
  <tr>
    <td>LWUP</td>
    <td>upwelling surface flux of longwave radiation</td>
  </tr>
</table>

Each of the RNET components is calculated using different data variables.

For each component we convert from units of J/m2 to W/m2 by differencing hourly values of ACSWDNB and dividing by the number of accumulated seconds.  For example, to compute SWDN in W/m2, we use:

\begin{equation}SWDN = (ACSWDNB[h] - ACSWDNB[h-1]) / 3600\end{equation}

which specifies values at the current hour (h) and the previous hour (h-1).

To code, first we define all values at time index h (the second timestep since indices start at 0)....

In [ ]:
ACSWDNB = c404_drb["ACSWDNB"][1:]

and then we define all values at time index h-1. Additionally we need to ensure that ACSWDNB and ACSWDNB1 have the same time indices.

In [ ]:
ACSWDNB1 = c404_drb["ACSWDNB"][:-1]
ACSWDNB1.coords["time"] = ACSWDNB.coords["time"]

Confirm both time coords are the same length.

In [ ]:
len(ACSWDNB.coords["time"].values) == len(ACSWDNB1.coords["time"].values)

Now calculate SWDN

In [ ]:
SWDN = (ACSWDNB - ACSWDNB1) / 3600

Let's visualize some of the results. Note the difference in time of day in the two maps.

In [ ]:
SWDN.sel(time="2000-06-01 10:00").hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

In [ ]:
SWDN.sel(time="2000-06-01 23:00").hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

Next, pad a NaN to the beginning to match original datasets dimension length and then reset to those dimensions

In [ ]:
SWDN = SWDN.pad({"time":(1,0)})
SWDN.coords["time"] = c404_drb["ACSWDNB"].coords["time"]

Now, we will do the same steps to calculate the other three components of *RNET*.

*SWUP*
\begin{equation}SWUP = (ACSWUPB[h] - ACSWUPB[h-1]) / 3600\end{equation}

In [ ]:
# (h) variables
ACSWUPB = c404_drb["ACSWUPB"][1:]

# (h-1) variables)
ACSWUPB1 = c404_drb["ACSWUPB"][:-1]
ACSWUPB1.coords["time"] = ACSWUPB.coords["time"]

# calculate variable
SWUP = (ACSWUPB - ACSWUPB1) / 3600

# pad to match c404_drb time dimension
SWUP = SWUP.pad({"time":(1,0)})
SWUP.coords["time"] = c404_drb["ACSWUPB"].coords["time"]

*LWDN*

\begin{equation}LWDN = (ACLWDNB[h] - ACLWDNB[h-1]) / 3600\end{equation}

In [ ]:
# (h) variables
ACLWDNB = c404_drb["ACLWDNB"][1:]

# (h-1) variables)
ACLWDNB1 = c404_drb["ACLWDNB"][:-1]
ACLWDNB1.coords["time"] = ACLWDNB.coords["time"]

# calculate variable
LWDN = (ACLWDNB - ACLWDNB1) / 3600

# pad to match c404_drb time dimension
LWDN = LWDN.pad({"time":(1,0)})
LWDN.coords["time"] = c404_drb["ACLWDNB"].coords["time"]

*LWUP*

\begin{equation}LWUP = (ACLWUPB[h] - ACLWUPB[h-1]) / 3600\end{equation}

In [ ]:
# (h) variables
ACLWUPB = c404_drb["ACLWUPB"][1:]

# (h-1) variables)
ACLWUPB1 = c404_drb["ACLWUPB"][:-1]
ACLWUPB1.coords["time"] = ACLWUPB.coords["time"]

# calculate variable
LWUP = (ACLWUPB - ACLWUPB1) / 3600

# pad to match c404_drb time dimension
LWUP = LWUP.pad({"time":(1,0)})
LWUP.coords["time"] = c404_drb["ACLWUPB"].coords["time"]

With all the components, calculate RNET...

In [ ]:
# calculate 
RNET = SWDN + LWDN - SWUP - LWUP
# RNET

...assign its attributes...

In [ ]:
# dictionary of attributes
RNET_attrs = {'description': 'SURFACE NET RADIATION',
 'grid_mapping': 'crs',
 'long_name': 'Surface net radiation',
 'units': 'W m-2'
}

# assign attributes
RNET = RNET.assign_attrs(RNET_attrs)
# RNET

 and assign it back to CONUS404

In [ ]:
c404_drb = c404_drb.assign(RNET=RNET)

Now drop the extra radiation variables

In [ ]:
c404_variables_drop = ["ACSWDNB", "ACSWUPB", "ACLWDNB", "ACLWUPB"]
c404_drb = c404_drb.drop_vars(c404_variables_drop)

Visualize RNET, which is accumulated net radiation in watts per m^2

In [ ]:
c404_drb["RNET"].sel(time="2000-06-01 23:00").hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

Our dataset has been clipped to the area of interest and all the needed variables calculated. The final bit of engineering is resampling the data from hourly to monthly. Xarray has a built in method `resample()` to do this but it only allows a single aggregation method for all the DataArrays in the DataSet. 

Unfortunately, the DataArrays need different aggregation techniques: we want to sum values of *PREC_ACC_NC*, but compute means of *RNET* and *TK*. We'll accomplish this by splitting *PREC_ACC_NC* from the dataset, resampling it and the dataset separately, then merging them back together.

In [ ]:
# copy data
PREC_ACC_NC = c404_drb["PREC_ACC_NC"]

# resample to 1 month by summing
PREC_ACC_NC = PREC_ACC_NC.resample(time="1M").sum()

# copy attributes from original
PREC_ACC_NC.attrs = c404_drb["PREC_ACC_NC"].attrs

# drop from c404_drb
c404_drb = c404_drb.drop_vars("PREC_ACC_NC")

Resample the dataset -- which now excludes PREC_ACC_NC -- and aggregate by mean.

In [ ]:
with xr.set_options(keep_attrs=True): # needed, otherwise drops attributes
    c404_drb = c404_drb.resample(time="1M").mean()

Add back the resampled *PREC_ACC_NC*

In [ ]:
c404_drb["PREC_ACC_NC"] = PREC_ACC_NC
# c404_drb

Correct attributes as needed

In [ ]:
c404_drb.PREC_ACC_NC.attrs["integration_length"] = 'accumulated over past month'
c404_drb.PREC_ACC_NC.attrs["grid_mapping"] = 'crs'
c404_drb.RNET.attrs["description"] = "MEAN RADIATION FROM PAST MONTH FOR BUCKET"
c404_drb.RNET.attrs["grid_mapping"] = 'crs'
c404_drb.TK.attrs["description"] = "MEAN AIR TEMPERATURE AT THE LOWEST MODEL LEVEL OVER THE PAST MONTH"
c404_drb.TK.attrs["grid_mapping"] = 'crs'

Drop coordinates that will not be need in the exported dataset

In [ ]:
c404_drb = c404_drb.reset_coords(["metpy_crs", "crs"], drop=True)

Add the crs data variable from the original CONUS404 dataset

In [ ]:
c404_drb['crs'] = ds_crs_array

Set chunk size so entire dataset is single chunk (the data is small enough)

In [ ]:
c404_drb = c404_drb.chunk(chunks={
    "y":103,
    "x":46,
    "time":492
})

Review the final prepared dataset

In [ ]:
c404_drb

Now it is time to export the data. For the purposes of this tutorial, you will be shown **how** to export the data to a requester pays (think permissioned) S3 bucket but will **not actually** export the data. The intermediate datasets created with the above and below code have been saved in a read-only space for the next two notebooks.

For working with climate and forcings data using the CF conventions, NETCDF is the format of choice.

First, `fsspec` is used to create an object to write the file to. Note that in the `open` method, it says `simplecache` when building the S3 file name. This has been found to be the easiest way to write files to a permissioned space using `fsspec`. `simplecache` works by locally caching the file before writing _that_ copy to S3 and then deleting the locally cached file.

Next, the object is written to using `xarray`'s built-in `to_netcdf` method. 

**Note**: running the code below will result in an error and not work.

In [ ]:
# outfile = fsspec.open('simplecache::s3://mybucket/c404_drb.nc', 
#                       mode='wb', s3=dict(profile='profile'))
# with outfile as f:
#     c404_drb.load().to_netcdf(f, compute=True)

## **Step 5: Prepare the gridded reference data**

Now that the CONUS404 dataset has been preprocessed, we need to do the same with datasets used for comparison with the forcings data. In this section, data will be brought in from several sources and preprocessed in data-type-appropriate ways.

We'll start by processing the rest of the gridded datasets.

#### PRISM V2 data
This time we will open the PRISM V2 dataset, temporarally slice it, spatially clip it, and refine the data. The PRISM V2 is daily data with a spatial resolution of 800 meters and we will be using the precipitation, maximum temperature, and minimum temperature data variables. The latter two will be used to calculate mean temperature. Finally, the PRISM V2 dataset is projected in the [NAD83 coordinate reference system](https://epsg.io/4269). Many of the steps will look the same as the CONUS404 dataset  so there will be less explanation of the steps. This dataset has been brought into the NHGF development server for storage but it is also available from the [PRISM Climate Group at Oregon State University](https://prism.oregonstate.edu/downloads/).

In [ ]:
# create filesystem object
fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

# range of years for data
prism_years = range(1979, 2021, 1)

# construct list of datasets for each year
chunks={"time": 6, "lon": 703, "lat": 311}
pr = [xr.open_dataset(fs.open(f"s3://nhgf-development/thredds/prism_v2/prism_{str(year)}.nc"), chunks=chunks, decode_coords="all") for year in prism_years]

# concat datasets
prism = xr.concat(pr, dim="time")

# drop time_bnds
prism = prism.drop_vars("time_bnds")

# set NAD83 with EPSG code 4269
prism_crs = 4269 

# write crs to prism
prism.rio.write_crs(prism_crs, inplace=True)

Rename the dimensions to match CF conventions used by rioxarray

In [ ]:
prism = prism.rename({"lon":"x", "lat":"y", "ppt": "PREC_ACC_NC"}) 

Bring in DRB boundaries with NAD83 crs...

In [ ]:
# bring in boundaries of DRB and create single polygon
drb_NAD83 = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])

# create a column where all entries have the same value and then...
drb_NAD83["name"] = "DRB"

# dissolve by that column
drb_NAD83 = drb_NAD83.dissolve(by="name")

# set CRS to match prism
drb_NAD83 = drb_NAD83.iloc[[0]].to_crs(prism_crs)

# get bounds of NAD83 drb
drb_NAD83_bounds = list(drb_NAD83.total_bounds)

#visualize
drb_NAD83.plot(edgecolor="orange", facecolor="purple", linewidth=2.5)

and perform clip to DRB extent

In [ ]:
# clip to DRB bbox
prism_drb = prism.rio.clip_box(minx=drb_NAD83_bounds[0],
                                  miny=drb_NAD83_bounds[1],
                                  maxx=drb_NAD83_bounds[2],
                                  maxy=drb_NAD83_bounds[3])

#slice time
prism_drb = prism_drb.sel(time=slice(start_date, end_date))
prism_drb

Display the clipped data

In [ ]:
prism_drb.TK.sel(time="2000-06-01", method="nearest").hvplot(x='x', y='y', rasterize=True, tiles='OSM', alpha=0.7, cmap='turbo')

Visualize the subset CONUS404 and PRISM data

In [ ]:
# [caelan] I think it could be nice here to have a comparison plot for conus 404 vs prism. 
# c404_drb["PREC_ACC_NC"].sel(time="2000-05-31", method = 'nearest').hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

Calculate the mean monthly tempertaure and convert to Kelvin and populate its attributes

In [ ]:
# mean temperature in Kelvin
prism_drb = prism_drb.assign(TK = ((prism_drb.tmn+prism_drb.tmx)/2) + 273.15) 

# dictionary of attributes
prism_tk_attrs = {'units': 'K',
 'long_name': 'Mean monthly temperature'
}

# assign attributes
prism_drb["TK"] = prism_drb["TK"].assign_attrs(prism_tk_attrs)

prism_drb.PREC_ACC_NC.attrs["units"] = 'mm'
prism_drb.PREC_ACC_NC.attrs["long_name"] = 'Accumulated grid scale precipitation'

# drop variables
prism_drb = prism_drb.drop_vars(["tmn", "tmx"])

# drop spatial_ref coord in order to export later
prism_drb = prism_drb.reset_coords("spatial_ref", drop=True)

# rechunk to make export easier
prism_drb = prism_drb.chunk(chunks={
    "y":98,
    "x":48,
    "time":492
})

# check out final dataset
prism_drb

Export data as a NetCDF file (this code is once again only an example and will result in an error if ran).

In [ ]:
# outfile = fsspec.open('simplecache::s3://mybucket/prism_drb.nc', 
#                       mode='wb', s3=dict(profile='profile'))
# with outfile as f:
#     prism_drb.load().to_netcdf(f, compute=True)

#### NASA's CERES-EBAF Level 3b Dataset
The [CERES-EBAF Summary](https://ceres.larc.nasa.gov/documents/DQ_summaries/CERES_EBAF_Ed4.1_DQS.pdf) provides important background information and insights about the data. The [download page](https://ceres.larc.nasa.gov/data/#ebaf-level-3) provides a quicks summary as well. The CERES-EBAF dataset includes radiation data at a monthy time step and at 1-degree latitude-longitude resolution.

In [ ]:
# bring in ceres-ebaf
ceres = xr.open_dataset("https://opendap.larc.nasa.gov/opendap/CERES/EBAF/Edition4.1/CERES_EBAF_Edition4.1_200003-202111.nc", decode_coords="all")

# parse_cf
ceres = ceres.metpy.parse_cf()

# extract crs
ceres_crs = ceres.sfc_net_tot_all_mon.metpy.cartopy_crs

# rename
ceres = ceres.rename({"lon":"x", "lat":"y", "sfc_net_tot_all_mon":"RNET"}) 

# x is in 0:360, need it in -180:180
ceres = ceres.assign_coords(x=(((ceres.x + 180) % 360) - 180)).sortby('x')

# pare down dataset 
ceres = ceres["RNET"]

# set crs
ceres.rio.write_crs(ceres_crs, inplace=True)

# bring in boundaries of DRB and create single polygon
drb_PC = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])

# create a column where all entries have the same value
drb_PC["name"] = "DRB"

# dissolve by that column
drb_PC = drb_PC.dissolve(by="name")

# set CRS to match ds
drb_PC = drb_PC.iloc[[0]].to_crs(ceres_crs)

# get bounds of NAD83 drb
drb_PC_bounds = list(drb_PC.total_bounds)

# clip to DRB bbox
ceres_drb = ceres.rio.clip_box(minx=drb_PC_bounds[0],
                                  miny=drb_PC_bounds[1],
                                  maxx=drb_PC_bounds[2],
                                  maxy=drb_PC_bounds[3])

#slice time
ceres_drb = ceres_drb.sel(time=slice(start_date, end_date))

# drop spatial_ref coord in order to export later
ceres_drb = ceres_drb.reset_coords(["metpy_crs"], drop=True)

#check final dataset
ceres_drb

Visualize the data

In [ ]:
ceres_drb.hvplot(x="x", y="y", rasterize=True)

Export data as a NetCDF file (this code is once again only an example and will result in an error if run).

In [ ]:
# outfile = fsspec.open('simplecache::s3://mybucket/ceres_drb.nc', 
#                       mode='wb', s3=dict(profile='profile'))
# with outfile as f:
#     ceres_drb.load().to_netcdf(f, compute=True)

## **Step 6: Prepare station reference data**

Now its on to the point datasets (stations).

#### NOAA's Global Historical Climate Network - Daily (GHCN) Dataset

The GHCN dataset contains station data for temperature, radiation, and precipitation data variables at a daily temporal resolution all from across the world. We are interested in a subset of these, the United States Historical Climate Network (USHCN). It is always important to review any readme or metadata files for the data you wish to bring in. The [GHCN readme](https://noaa-ghcn-pds.s3.amazonaws.com/readme.txt) is useful because it explains what is in the S3 bucket, the various columns in the datasets, and other information. When we later call in the observational data, the [by station readme](https://noaa-ghcn-pds.s3.amazonaws.com/readme-by_station.txt) provides a more detailed explanation of the data there.

The steps for working with the GHCN data will be that we first read in the data that describe the stations (metadata); then we'll read in time-series data for each station.

After reading the metadata for the file, we can see that only the first three columns are needed to map the stations: the station ID, latitude, and longitude. However, we want to make sure that we are only using USHCN stations so we need to also use the (G)HCN/CRN Flag column to filter to USHCN sites. 

Start by getting a list of stations from the AWS S3 bucket where the daily data is housed.

In [ ]:
ghcn_all = pd.read_csv('s3://noaa-ghcn-pds/ghcnd-stations.txt', sep="\t", header=None)
ghcn_all.head(2)

As you can see, the read does not recognize the file's columns and the data are stored as one column per record. So, we have to split the record to create the columns that we need.

In [ ]:
ghcn_all = ghcn_all[0].str.split(" +",expand = True)
ghcn_all.head(2)

Columns 0-3 now look as we'd expect. However, column 4 is where it starts to get messy as the method for expanding the columns has split up the station names at the spaces between. This means that the HCN flag, which we would expect to be in column 6, could be in columns 6-13. Thankfully the pandas ```loc``` function handles this problem easily so that we can successfully filter just the 'HCN' stations. Note that the | represents the "or" logic and will return any row that matches any of the conditions. 

In [ ]:
ushcn = ghcn_all.loc[(ghcn_all[6] == "HCN") | (ghcn_all[7] == "HCN") | (ghcn_all[8] == "HCN") | (ghcn_all[9] == "HCN") | (ghcn_all[10] == "HCN") | (ghcn_all[11] == "HCN") | (ghcn_all[12] == "HCN") | (ghcn_all[13] == "HCN")].copy()
ushcn = ushcn.iloc[:, 0:3].rename({0:"station", 1:"lat", 2:"lon"}, axis=1).copy() # after the search, trim the columns and rename to get the data to what is needed to map
ushcn.head()

We now need to clip the points to only those in the DRB. We do that by using the latitude and longitude to create a GeoDataFrame...

In [ ]:
ushcn_gdf = gpd.GeoDataFrame(ushcn, geometry=gpd.points_from_xy(ushcn['lon'], ushcn['lat'], crs="EPSG:4326"))

# convert to same crs as drb
ushcn_gdf = ushcn_gdf.to_crs(crs)

# ushcn_gdf.plot()

...followed by clipping using the *drb* geodataframe above.

In [ ]:
hcn_drb_gdf = gpd.clip(ushcn_gdf, drb)
# hcn_drb_gdf.plot()

Now we want to pull in the tabular data for all of the DRB stations indexed by time. These are stored on AWS in an individual CSV file for each station named *station.csv*. So, we need to get the station IDs from our DRB subset in order to create a list of URLs for the corresponding csv files.

In [ ]:
hcn_drb_data_url = [f"s3://noaa-ghcn-pds/csv/by_station/{station}.csv" for station in hcn_drb_gdf["station"].unique().tolist()]
# print(hcn_drb_data_url[0:3])

In [ ]:
# len(hcn_drb_data_url)

We'll now pass that list of URLs to *dask.dataframe.read_csv*, which will read the data files in parallel. We'll then refine the entries to the temporal range of CONUS404.

In [ ]:
storage_options = dict(anon=True, requester_pays=False)
hcn_drb_data = dd.read_csv(hcn_drb_data_url, parse_dates=["DATE"], 
                           usecols=["ID", "DATE", "ELEMENT", "DATA_VALUE"], storage_options=storage_options)

hcn_drb_data = hcn_drb_data.loc[(hcn_drb_data["DATE"] >= start_date) & (hcn_drb_data["DATE"] <= end_date)]

Next, we'll refine the dataframe by a list of elements and then compute it, which is a `dask` method for reading the dataset into memory.

##### Note: We are using TMAX and TMIN rather than TAVG as TAVG has no records prior to 1998.

In [ ]:
# list of elements we are interested in
element_list = ["PRCP", "TMAX", "TMIN"]

hcn_drb_data = hcn_drb_data.loc[hcn_drb_data["ELEMENT"].isin(element_list)]

In [ ]:
# check shape
hcn_drb_data.compute().shape

In [ ]:
# how much memory does it take up?
# hcn_drb_data.compute().memory_usage()

The dask dataframe is about 21 mb in size.

Similar to the CONUS404 data, we have a little more engineering to do with the data. We need to calculate the average temperatue using TMIN and TMAX (in Kelvin) as well as resample the data from a 1 day to a 1 month interval. We'll convert the Dask Dataframe into a Pandas Dataframe to do this.

In [ ]:
hcn_drb_data_df = hcn_drb_data.compute()

We start by whittling down to our two temperature elements, dropping the *ELEMENT* column, and grouping our data by *ID* and *DATE* in order to take the mean of *TMIN* and *TMAX* and convert this to degrees Kelvin.

In [ ]:
# paring down data
hcn_drb_tk = hcn_drb_data_df.loc[hcn_drb_data_df["ELEMENT"].isin(["TMAX","TMIN"])]

# dropping ELEMENT
hcn_drb_tk = hcn_drb_tk.drop("ELEMENT", axis=1)

# calculate mean temperature for each station and date
hcn_drb_tk = hcn_drb_tk.groupby(["ID", "DATE"]).mean()

# rename the DATA_VALUE column to TK
hcn_drb_tk = hcn_drb_tk.rename({"DATA_VALUE":"TK"}, axis=1)

# convert from tenths of degrees Celsius to degrees Kelvin
hcn_drb_tk["TK"] = (hcn_drb_tk["TK"] * 0.1) + 273.15

# reset the index
hcn_drb_tk.reset_index(inplace=True)

Isolate the *PRCP* element, rename for consistency, and convert from units of tenths of mm to mm.

In [ ]:
hcn_drb_prcp = hcn_drb_data_df.loc[hcn_drb_data_df["ELEMENT"] == "PRCP"].copy()

# dropping ELEMENT
hcn_drb_prcp = hcn_drb_prcp.drop("ELEMENT", axis=1)

# rename the DATA_VALUE column to PREC_ACC_NC
hcn_drb_prcp = hcn_drb_prcp.rename({"DATA_VALUE":"PREC_ACC_NC"}, axis=1)

# convert from tenths of mm to mm
hcn_drb_prcp["PREC_ACC_NC"] = hcn_drb_prcp["PREC_ACC_NC"] * 0.1

# reset the index
hcn_drb_prcp.reset_index(inplace=True, drop=True)

# hcn_drb_prcp

Combine *TK* and *PRCP* DataFrames

In [ ]:
hcn_drb = hcn_drb_tk.merge(hcn_drb_prcp, how="inner", on=["ID", "DATE"])
# hcn_drb

And then resample to 1 month and aggregate

In [ ]:
hcn_drb = hcn_drb.groupby("ID").resample("1M", on="DATE").agg({"TK":"mean", "PREC_ACC_NC":"sum"}).reset_index(drop=False)

# round TK
hcn_drb.TK = round(hcn_drb.TK, 2)
# hcn_drb

Now add the latitude and longitude coordinates 

In [ ]:
hcn_drb_coords = pd.DataFrame(hcn_drb_gdf.drop(columns="geometry"))

# rename ID columnt o match drb_hcn
hcn_drb_coords = hcn_drb_coords.rename({"station": "ID", "lon":"LONGITUDE", "lat":"LATITUDE"}, axis=1)

# merge
hcn_drb = hcn_drb.merge(hcn_drb_coords, on="ID", how="left")

hcn_drb.head()

Exporting the station data will be a bit different. First, the data type is different as we used the `xarray` dataset for the gridded data and a `pandas` dataframe for the station. Since our station data is essentially column-oriented tabular data, a good file type for storage is [parquet](https://parquet.apache.org/). An additional benefit is that `pandas` uses `fsspec` in the background to handle IO so it will use environmental variables such as an AWS ID and Secret Key to handle writing to many permissioned spaces. An example of this would look like (this is an example and will result in an error if run):

In [ ]:
# hcn_drb.to_parquet("s3://file/path/to/hcn_drb.parquet")

#### NOAA's Global Climate Reference Network (GCRN) Dataset

For the GCRN, we will be using the accumulated precipitation and average temperature variables. The GCRN will be subset to United States Climate Reference Network (USCRN) stations. The documentation for the GCRN dataset can be found at [the FTP site](https://www.ncei.noaa.gov/pub/data/uscrn/products/monthly01/readme.txt). As this data will be brought in over an FTP call, the process will be a bit different than the other datasets, using *fsspec* to make FTP call to NOAA for CRN data. <br> 

First, create a FTP file system.

In [ ]:
fs = FTPFileSystem("ftp.ncei.noaa.gov")

Since the file type is *tab-separated values (tsv)*, we will use the *pd.read_table* function to create a Dataframe

In [ ]:
uscrn_all = pd.read_table(fs.open("/pub/data/uscrn/products/stations.tsv")) 
uscrn_all.head()

Now turn into GeoDataFrame.

In [ ]:
uscrn_gdf = gpd.GeoDataFrame(uscrn_all, geometry=gpd.points_from_xy(uscrn_all["LONGITUDE"], uscrn_all["LATITUDE"]), crs="EPSG:4326")

# convert to same crs as drb
uscrn_gdf = uscrn_gdf.to_crs(crs)

# uscrn_gdf.plot()

Find which USCRN sites are in DRB

In [ ]:
crn_drb_gdf = gpd.clip(uscrn_gdf, drb)
# crn_drb_gdf.plot()

In [ ]:
crn_drb_gdf.head()

We now would know what CRN sites are in the Delaware River Basin. However, note that the USCRN station network is much sparser than GHCN and only one USCRN station is located in the DRB. We must now retrieve the data for this site from the FTP server.

First, we'll get the location name.

In [ ]:
crn_stat_name = crn_drb_gdf["LOCATION"].values.tolist()[0]
print(crn_stat_name)

Initilize the FTP connection again

In [ ]:
fs = FTPFileSystem("ftp.ncei.noaa.gov")

In [ ]:
file_list_glob = fs.glob(
    f"/pub/data/uscrn/products/daily01/**/*{crn_stat_name}*")

# see list of files
# for file in file_list_glob:
#     print(file,"\n")

In [ ]:
crn_drb_dfs = [pd.read_csv(fs.open(file), header=None, sep="\t") for file in file_list_glob]

crn_drb = pd.concat(crn_drb_dfs)

# split single column into all
crn_drb = crn_drb[0].str.split(" +",expand = True)

Now bring in the headers for the station data

In [ ]:
crn_headers = fs.open("/pub/data/uscrn/products/daily01/headers.txt")
crn_data_headers = pd.read_csv(crn_headers, sep="\t", header=None).iloc[1,:].str.split(" +").values.tolist()[0][0:28]
# crn_data_headers

Check that the number of headers equals the number of columns in our data

In [ ]:
print(len(crn_drb.columns) == len(crn_data_headers))

Now rename the column headers

In [ ]:
crn_drb.columns = crn_data_headers
# crn_drb

The *LST_DATE* column is the date of the observation and its data type is currently string. We want the data type to be [datetime](https://docs.python.org/3/library/datetime.html) so we will perform this conversion using the `to_datetime` function from the `pandas` library then refine the columns.

In [ ]:
crn_drb["DATE"] = pd.to_datetime(crn_drb["LST_DATE"])
crn_drb = crn_drb[["DATE", "P_DAILY_CALC", "T_DAILY_AVG", "LONGITUDE", "LATITUDE"]]

If you examine the data types, you'll see that the 4 columns of numbers are actually data type *object* when we need them as numeric

In [ ]:
crn_drb.dtypes

Lets rectify that by applying the `pd.to_numeric` function to the columns.

In [ ]:
cols = crn_drb.columns.drop("DATE")
crn_drb[cols] = crn_drb[cols].apply(pd.to_numeric, errors='coerce')
crn_drb.dtypes

Now refine the data by date

In [ ]:
crn_drb = crn_drb.loc[(crn_drb["DATE"] >= start_date) & (crn_drb["DATE"] <= end_date)]
print(crn_drb.shape)

The CRN dataset has many values of -9999.0, which is where a record was not recorded due to data quality or other issues. 

In [ ]:
crn_drb.head(2)

Because the value -9999.0 is seen as a valid number, set these values to NaN so they will be ignored during calculations.

In [ ]:
# set to NaN
crn_drb = crn_drb.replace(-9999.0, np.nan)

print(crn_drb.shape)

Now convert columns to the correct units

In [ ]:
# Celsius to Kelvin
crn_drb["TK"] = crn_drb["T_DAILY_AVG"] + 273.15

# drop columns
crn_drb = crn_drb.drop(["T_DAILY_AVG"], axis=1)

# rename column
crn_drb = crn_drb.rename({"P_DAILY_CALC": "PREC_ACC_NC"}, axis=1)

print(crn_drb.shape)

Resample the data and round TK

In [ ]:
crn_drb = crn_drb.resample("1M", on="DATE").agg({"TK":"mean", 
                                               "PREC_ACC_NC":"sum",
                                               "LATITUDE":"mean", 
                                               "LONGITUDE":"mean"}).reset_index(drop=False)

# round 
crn_drb["TK"] = round(crn_drb["TK"], 2)


# add a station ID column and modify DATE to be YYYY-dd
crn_drb["ID"] = crn_stat_name

crn_drb

Export the dataset (this is an example only and will result in an error if run).

In [ ]:
# crn_drb.to_parquet("s3://file/path/to/crn_drb.parquet")

And the final, always important step of shutting down the 'dask' client and cluster.

In [ ]:
client.close(); cluster.shutdown()

## Wrapping Up
Congratulations, you made it! I know this notebook contained a lot but it serves as a practical tutorial to see how diverse data input, preparation, and data output methods are handled. The best analogy I can think of is thinking about painting: 85% of the work is preparation. 

The next notebook will walk you through analysing CONUS404 against the reference data sets and include topics such as area weighted zonal statistics, point extraction of gridded data, spatial statistics, and temporal statistics.